In [1]:
import numpy as np
import h5py

from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score

from neuropacks import ECOG

In [2]:
ecog = ECOG(data_path='/Users/psachdeva/data/ECoG/data/r32_b7.mat')

In [4]:
ecog.freq_set

array([  500.,   577.,   666.,   769.,   887.,  1024.,  1182.,  1364.,
        1575.,  1818.,  2098.,  2421.,  2795.,  3226.,  3723.,  4297.,
        4960.,  5725.,  6608.,  7627.,  8803., 10160., 11727., 13535.,
       15622., 18031., 20812., 24021., 27725., 32000.])

In [3]:
Y = ecog.get_response_matrix(bounds=(40,60), band='HG')

In [6]:
f = h5py.File('fits/ecog/ecog.h5', 'r')

In [12]:
electrode = 3
fold = 5

In [14]:
training_idx = f['Lasso_HG/training_folds/fold_' + str(fold)][:]
test_idx = f['Lasso_HG/test_folds/fold_' + str(fold)][:]

X_train = np.delete(Y[training_idx], electrode, axis=1)
y_train = Y[training_idx][:, electrode]

X_test = np.delete(Y[test_idx], electrode, axis=1)
y_test = Y[test_idx][:, electrode]

In [17]:
lasso = LassoCV(normalize=True, cv=10, max_iter=5000)
lasso.fit(X_train, y_train)

LassoCV(alphas=None, copy_X=True, cv=10, eps=0.001, fit_intercept=True,
    max_iter=5000, n_alphas=100, n_jobs=None, normalize=True,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)

In [19]:
print(lasso.intercept_)
print(f['Lasso_HG/intercepts'][fold, electrode])

-0.0031951507322683925
-0.0031951507322685035


In [20]:
print(lasso.coef_)
print(f['Lasso_HG/coupling_coefs'][fold, electrode, :])

[ 3.05796914e-02  2.45492973e-01 -1.85636018e-01 -3.68981798e-02
  2.02237392e-01  0.00000000e+00 -6.58040425e-02  5.44622044e-02
 -0.00000000e+00  6.45688045e-01  3.43159453e-01 -8.61067286e-02
 -7.86121615e-02  2.00398483e-02  5.16053314e-02  0.00000000e+00
 -1.09694287e-03  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -1.69780428e-02  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -2.17631427e-02  0.00000000e+00
 -1.82198506e-02  0.00000000e+00 -0.00000000e+00  9.59379526e-03
  0.00000000e+00  0.00000000e+00  1.09650652e-03  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -7.94392555e-04  0.00000000e+00
 -0.00000000e+00  5.47132012e-03  0.00000000e+00  0.00000000e+00
  3.29671102e-04  0.00000000e+00  3.60081838e-04  0.00000000e+00
  0.00000000e+00  1.51435239e-03  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -2.12089218e-03
 -9.74814664e-03 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000

In [21]:
print(r2_score(y_test, np.dot(X_test, lasso.coef_) + lasso.intercept_))
print(f['Lasso_HG/r2'][fold, electrode])

0.9777391054751166
0.9777391054751166
